In [1]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [6]:
chat = ChatOpenAI(
    model = "gpt-3.5-turbo",
)

prompt = PromptTemplate(
    template = "{product}と{product2}はどこの会社が開発した製品ですか？",
    input_variables = ["product", "product2"]
)

result = chat([
    SystemMessage(content="あなたは親しい友人です。返答は敬語を使わず、フランクに会話してください。"),
    HumanMessage(content=prompt.format(product="iPhone", product2="プリウス")),
])

In [8]:
print(result.content)

iPhoneはAppleが開発した製品で、プリウスはToyotaが開発した製品だよ。どちらも人気があるね！


# プロンプトをJSONファイルに保存＆呼び出し

In [9]:
prompt_json = prompt.save("prompt.json")

In [10]:
from langchain.prompts import load_prompt
loaded_prompt = load_prompt("prompt.json")
print(loaded_prompt.format(product="Suica", product2="Pasmo"))

SuicaとPasmoはどこの会社が開発した製品ですか？


# リスト形式で結果を受け取る

In [13]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser() # 初期化

chat = ChatOpenAI(model = "gpt-3.5-turbo")

prompt = PromptTemplate(
    template = "{company}が開発した代表的な製品を5つ教えてください",
    input_variables = ["company"]
)

result = chat([
    SystemMessage(content="あなたはとても優秀なAIです。"),
    HumanMessage(content=prompt.format(company="トヨタ")),
    HumanMessage(content=output_parser.get_format_instructions()),
])

In [16]:
output_parser.parse(result.content)

['Prius', 'Corolla', 'Camry', 'RAV4', 'Land Cruiser']

# LLMsを使って文章の続きを予測

In [19]:
from langchain.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")

result = llm(
    "今日は朝ごはんに",
    stop="。"
)

print(result)

おにぎりを食べました


# 結果を逐次表示させる

In [22]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model = "gpt-3.5-turbo",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

resp = chat([
    HumanMessage(content="おいしいたこ焼きの作り方を教えて")
])

たこ焼きの作り方は以下の通りです：

材料：
- たこ焼き粉 200g
- だし汁 400ml
- 卵 1個
- たこ（小口切り） 100g
- 紅ショウガ（刻んだもの） 適量
- 青のり 適量
- かつお節 適量
- 天かす 適量
- ソース（お好みで）

作り方：
1. ボウルにたこ焼き粉を入れ、だし汁を少しずつ加えながら混ぜる。卵も加えてよく混ぜる。
2. たこや紅ショウガを入れ、全体を混ぜる。
3. たこ焼き器を予熱し、少量の油を塗る。
4. 混ぜた生地をたこ焼き器の穴に流し入れ、たこ焼きを焼く。
5. たこ焼きが焼けたら、竹串などでひっくり返し、裏返して焼く。
6. たこ焼きがふっくらと焼けたら、青のり、かつお節、天かすなどをトッピングする。
7. ソースをかけて完成。

お好みでマヨネーズや青海苔などをトッピングしても美味しいです。ぜひお試しください！

In [24]:
response_text = resp.content
response_text

'たこ焼きの作り方は以下の通りです：\n\n材料：\n- たこ焼き粉 200g\n- だし汁 400ml\n- 卵 1個\n- たこ（小口切り） 100g\n- 紅ショウガ（刻んだもの） 適量\n- 青のり 適量\n- かつお節 適量\n- 天かす 適量\n- ソース（お好みで）\n\n作り方：\n1. ボウルにたこ焼き粉を入れ、だし汁を少しずつ加えながら混ぜる。卵も加えてよく混ぜる。\n2. たこや紅ショウガを入れ、全体を混ぜる。\n3. たこ焼き器を予熱し、少量の油を塗る。\n4. 混ぜた生地をたこ焼き器の穴に流し入れ、たこ焼きを焼く。\n5. たこ焼きが焼けたら、竹串などでひっくり返し、裏返して焼く。\n6. たこ焼きがふっくらと焼けたら、青のり、かつお節、天かすなどをトッピングする。\n7. ソースをかけて完成。\n\nお好みでマヨネーズや青海苔などをトッピングしても美味しいです。ぜひお試しください！'

# プロンプトエンジニアリング

In [26]:
from langchain.llms import OpenAI
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [27]:
examples = [{
    "input": "LangChainはLLMの実利用をより柔軟に簡易に行うためのツール群です",
    "output": "LangChainは、LLMの実利用をより柔軟に、簡易に行うためのツール群です。"
}]

prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="入力: {input}\n出力: {output}",
)

few_shot_prompt = FewShotPromptTemplate(
    ## 入力例と出力例を定義
    examples=examples,
    ## プロンプトテンプレートを指定
    example_prompt=prompt,
    ## 指示を追加
    prefix="以下の句読点の抜けた入力に句読点を追加してください。追加して良い句読点は「、」「。」のみです。他の句読点は追加しないでください。",
    ## 出力例の入力変数を定義
    suffix="入力: {input_string}\n出力:",
    ## FewShotPromptTemplateの入力変数を設定
    input_variables=["input_string"],
)

llm = OpenAI()

In [30]:
formatted_prompt = few_shot_prompt.format(
    input_string="私は今日木曽路に行ってお昼ご飯を家族で食べますその後に息子と電車に乗って錦糸町に行きます"
)

result = llm.predict(formatted_prompt)

In [31]:
print("formatted_prompt: ", formatted_prompt)
print("result: ", result)

formatted_prompt:  以下の句読点の抜けた入力に句読点を追加してください。追加して良い句読点は「、」「。」のみです。他の句読点は追加しないでください。

入力: LangChainはLLMの実利用をより柔軟に簡易に行うためのツール群です
出力: LangChainは、LLMの実利用をより柔軟に、簡易に行うためのツール群です。

入力: 私は今日木曽路に行ってお昼ご飯を家族で食べますその後に息子と電車に乗って錦糸町に行きます
出力:
result:   私は、今日木曽路に行って、お昼ご飯を家族で食べます。その後に、息子と電車に乗って、錦糸町に行きます。


# 結果を日時形式で受け取る

In [32]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import DatetimeOutputParser # 出力を日時形式に変換するモジュール
from langchain.schema import HumanMessage

In [45]:
output_parser = DatetimeOutputParser() # DatetimeOutputParserを初期化

chat = ChatOpenAI(model="gpt-4")
prompt = PromptTemplate.from_template("{product}の発売日を教えて")
result = chat([
    HumanMessage(content=prompt.format(product="Nintendo switch")),
    HumanMessage(content=output_parser.get_format_instructions()),
])

output = output_parser.parse(result.content)
print(output)

2017-03-03 00:00:00


# 出力形式を自分で定義する（エラーが出た場合に修正させる）

In [58]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import OutputFixingParser
from langchain.output_parsers import PydanticOutputParser
from langchain.schema import HumanMessage
from pydantic import BaseModel, Field, field_validator

In [83]:
chat = ChatOpenAI(model="gpt-4")

class Smartphone(BaseModel):
    release_date: str = Field(description="スマートフォンの発売日")
    screen_inches: float = Field(description="スマートフォンの画面サイズ（インチ）")
    os_installed: str = Field(description="スマートフォンにインストールされているOS")
    modelname: str = Field(description="スマートフォンのモデル名")

    @field_validator("screen_inches") # validatorを使って値を検証
    def validate_screen_inches(cls, field):
        if field <= 0: # screen_inchesが0以下の場合はエラーを出す
            raise ValueError("Screen inches must be a positive number")
        return field

### エラーが出力されたときに修正を指示
parser = OutputFixingParser.from_llm(
    # PydanticOutputParserをSmartphoneクラスで初期化
    parser = PydanticOutputParser(pydantic_object=Smartphone),
    # 修正に使用する言語モデルを設定
    llm = chat,
)

# parser = PydanticOutputParser(pydantic_object=Smartphone)

result = chat([
    HumanMessage(content="windowsでリリースしたスマートフォンを1個挙げて"),
    HumanMessage(content=parser.get_format_instructions()),
])

In [84]:
parsed_result = parser.parse(result.content)
print(f"モデル名: {parsed_result.modelname}")
print(f"画面サイズ: {parsed_result.screen_inches}インチ")
print(f"OS: {parsed_result.os_installed}")
print(f"発売日: {parsed_result.release_date}")

モデル名: Lumia 950
画面サイズ: 5.2インチ
OS: Windows 10 Mobile
発売日: 2015-10-06
